# VITS-official
[![Package badge]][github]
[![Open In Colab]][notebook]

Author: VITS OSS group & [tarepan]

[github]:https://github.com/tarepan/VITS-official
[notebook]:https://colab.research.google.com/github/tarepan/VITS-official/blob/main/inference.ipynb
[tarepan]:https://github.com/tarepan
[Package badge]:https://img.shields.io/badge/GitHub-vits-9cf.svg
[Open In Colab]:https://colab.research.google.com/assets/colab-badge.svg

## Setup

In [1]:
# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Clone the repository
!git clone https://github.com/tarepan/VITS-official
%cd VITS-official

Cloning into 'VITS-official'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 85 (delta 0), reused 1 (delta 0), pack-reused 81
Receiving objects: 100% (85/85), 3.34 MiB | 8.31 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/VITS-official


In [3]:
# Install dependencies
!pip install phonemizer==2.2.1 Unidecode==1.1.1
!apt-get install espeak

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.3/238.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 33.1 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136818 sha256=6f7f2af4f1b1239431176e1f61e0f55128e816bb8fd34a1b3c7ecf577dc34bfe
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
Reading package lists

In [4]:
# Build internal dependencies
%cd monotonic_align
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..

/content/VITS-official/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/content/VITS-official


In [5]:
# Copy pretrained model
!cp /content/gdrive/MyDrive/ML_data/pretrained/VITS_official/pretrained_ljs.pth ./
!cp /content/gdrive/MyDrive/ML_data/pretrained/VITS_official/pretrained_vctk.pth ./

## Inference

### TTS

#### Setup

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

mode = "LJ"
# mode = "VCTK"
hps = utils.get_hparams_from_file("./configs/vctk_base.json" if mode == "VCTK" else "./configs/ljs_base.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers if  mode == "VCTK" else 0,
    **hps.model)
# net_g = net_g.cuda()
net_g.eval()
utils.load_checkpoint("./pretrained_vctk.pth" if mode == "VCTK" else "./pretrained_ljs.pth", net_g, None)

(SynthesizerTrn(
   (enc_p): TextEncoder(
     (emb): Embedding(178, 192)
     (encoder): Encoder(
       (drop): Dropout(p=0.1, inplace=False)
       (attn_layers): ModuleList(
         (0-5): 6 x MultiHeadAttention(
           (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (drop): Dropout(p=0.1, inplace=False)
         )
       )
       (norm_layers_1): ModuleList(
         (0-5): 6 x LayerNorm()
       )
       (ffn_layers): ModuleList(
         (0-5): 6 x FFN(
           (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
           (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
           (drop): Dropout(p=0.1, inplace=False)
         )
       )
       (norm_layers_2): ModuleList(
         (0-5): 6 x LayerNorm()
       )
     )
     (proj)

#### Run

In [7]:
stn_tst = get_text("VITS is Awesome! Panda!", hps)

with torch.no_grad():
    x_tst         = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    sid           = torch.LongTensor([4])
    # x_tst, x_tst_lengths, sid = x_tst.cuda(), x_tst_lengths.cuda(), sid.cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()

ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

### VC

In [8]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

FileNotFoundError: ignored

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))